- 應該要裝 mistralai openai tiktoken ipywidgets python-dotenv
- 然後還要改 pdf2md 的 generate_summary 或 analysis 要變成 通順化 / 翻譯版

## batch pdf to md

In [10]:
import os
import ipywidgets as widgets
from IPython.display import display
from pdf2md import pdf2md  # 請確認 pdf2md 函式的引入路徑正確

def list_pdf_files(input_folder):
    """列出指定資料夾內所有 PDF 檔案的完整路徑"""
    pdf_files = []
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(input_folder, filename))
    return pdf_files

def batch_convert_pdfs(input_folder, output_folder, requirement=""):
    pdf_files = list_pdf_files(input_folder)
    
    if not pdf_files:
        print("在資料夾中找不到任何 PDF 檔案！")
        return
    
    # 為每個 PDF 建立一列設定的 widget
    file_widgets = []
    for path in pdf_files:
        basename = os.path.basename(path)
        # 每個檔案有四個選項：
        # process_checkbox：是否要處理該檔案（預設 True）
        # need_summary_checkbox：是否啟用 need_summary（預設 True）
        # need_preview_checkbox：是否啟用 need_preview（預設 True）
        # need_writing_analysis_checkbox：是否啟用 need_writing_analysis（預設 False）
        # requirement_text：使用者輸入的 requirement（預設為空字串）
        process_checkbox = widgets.Checkbox(value=True, description="轉檔", indent=False)
        need_summary_checkbox = widgets.Checkbox(value=True, description="need_summary", indent=False)
        need_preview_checkbox = widgets.Checkbox(value=True, description="need_preview", indent=False)
        need_writing_analysis_checkbox = widgets.Checkbox(value=False, description="need_writing_analysis", indent=False)
        requirement_text = widgets.Text(value=requirement, placeholder="輸入 requirement", description="Requirement:", layout=widgets.Layout(width="250px"))
        
        row = widgets.HBox([
            widgets.Label(basename, layout=widgets.Layout(width="250px")),
            process_checkbox,
            need_summary_checkbox,
            need_preview_checkbox,
            need_writing_analysis_checkbox,
            requirement_text
        ])
        file_widgets.append((path, row, process_checkbox, need_summary_checkbox, need_preview_checkbox, need_writing_analysis_checkbox, requirement_text))
    
    # 將所有檔案的 widget 放到一個 VBox 中
    files_vbox = widgets.VBox([row for (_, row, _, _, _, _, _) in file_widgets])
    
    # 建立確認與取消按鈕以及輸出區域
    confirm_button = widgets.Button(description="開始轉檔", button_style="success")
    cancel_button = widgets.Button(description="取消轉檔", button_style="danger")
    output_area = widgets.Output()
    
    def on_confirm_clicked(b):
        with output_area:
            print("開始轉檔...\n")
            for (path, row, process_checkbox, need_summary_checkbox, need_preview_checkbox, need_writing_analysis_checkbox, requirement_text) in file_widgets:
                if process_checkbox.value:
                    pdf_name = os.path.splitext(os.path.basename(path))[0]
                    
                    os.makedirs(output_folder, exist_ok=True)
                    print(f"轉檔：{os.path.basename(path)}")
                    try:
                        # 傳入 requirement_text.value 作為 requirement 參數
                        pdf2md(
                            path, 
                            output_folder, 
                            need_summary=need_summary_checkbox.value, 
                            need_preview=need_preview_checkbox.value, 
                            need_writing_analysis=need_writing_analysis_checkbox.value,
                            requirement=requirement_text.value
                        )
                        print(f"轉檔完成，輸出資料夾：{output_folder}\n")
                    except Exception as e:
                        print(f"轉檔 {os.path.basename(path)} 時發生錯誤：{e}\n")
                else:
                    print(f"跳過：{os.path.basename(path)}\n")
    
    def on_cancel_clicked(b):
        with output_area:
            print("已取消轉檔。")
    
    confirm_button.on_click(on_confirm_clicked)
    cancel_button.on_click(on_cancel_clicked)
    
    # 組合所有 widget 並顯示
    display(widgets.VBox([
        widgets.Label("找到以下 PDF 檔案（每個檔案可個別設定）："),
        files_vbox,
        widgets.HBox([confirm_button, cancel_button]),
        output_area
    ]))


In [8]:
input_folder = ".\inputs"
output_folder = ".\outputs"
requirement = ""
batch_convert_pdfs(input_folder, output_folder, requirement)

## batch md analysis

In [ ]:
import os
import time
import ipywidgets as widgets
from IPython.display import display
from pdf2md import md_analysis  # 請確認 md_analysis 的引入路徑正確

def list_md_files(input_folder):
    """列出指定資料夾內所有 Markdown 檔案的完整路徑"""
    md_files = []
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".md"):
            md_files.append(os.path.join(input_folder, filename))
    return md_files

def batch_md_analysis(input_folder, output_folder):
    md_files = list_md_files(input_folder)
    
    if not md_files:
        print("在資料夾中找不到任何 Markdown 檔案！")
        return
    
    # 為每個 Markdown 檔案建立一列設定的 widget
    file_widgets = []
    for path in md_files:
        basename = os.path.basename(path)
        # 每個檔案有以下選項：
        # process_checkbox：是否要處理該檔案（預設 True）
        # need_summary_checkbox：是否啟用 need_summary（預設 True）
        # need_preview_checkbox：是否啟用 need_preview（預設 True）
        # need_writing_analysis_checkbox：是否啟用 need_writing_analysis（預設 False）
        # requirement_text：使用者輸入的 requirement（預設空字串）
        process_checkbox = widgets.Checkbox(value=True, description="處理", indent=False)
        need_summary_checkbox = widgets.Checkbox(value=True, description="need_summary", indent=False)
        need_preview_checkbox = widgets.Checkbox(value=True, description="need_preview", indent=False)
        need_writing_analysis_checkbox = widgets.Checkbox(value=False, description="need_writing_analysis", indent=False)
        requirement_text = widgets.Text(value="", placeholder="輸入 requirement", description="Requirement:", layout=widgets.Layout(width="250px"))
        
        row = widgets.HBox([
            widgets.Label(basename, layout=widgets.Layout(width="250px")),
            process_checkbox,
            need_summary_checkbox,
            need_preview_checkbox,
            need_writing_analysis_checkbox,
            requirement_text
        ])
        file_widgets.append((path, row, process_checkbox, need_summary_checkbox, need_preview_checkbox, need_writing_analysis_checkbox, requirement_text))
    
    # 將所有檔案的 widget 放到一個 VBox 中
    files_vbox = widgets.VBox([row for (_, row, _, _, _, _, _) in file_widgets])
    
    # 建立確認與取消按鈕以及輸出區域
    confirm_button = widgets.Button(description="開始分析", button_style="success")
    cancel_button = widgets.Button(description="取消分析", button_style="danger")
    output_area = widgets.Output()
    
    def on_confirm_clicked(b):
        with output_area:
            print("開始分析...\n")
            for (path, row, process_checkbox, need_summary_checkbox, need_preview_checkbox, need_writing_analysis_checkbox, requirement_text) in file_widgets:
                if process_checkbox.value:
                    md_name = os.path.splitext(os.path.basename(path))[0]
                    
                    os.makedirs(output_folder, exist_ok=True)
                    print(f"分析：{os.path.basename(path)}")
                    try:
                        md_analysis(
                            path, 
                            output_folder, 
                            need_summary=need_summary_checkbox.value, 
                            need_preview=need_preview_checkbox.value, 
                            need_writing_analysis=need_writing_analysis_checkbox.value,
                            requirement=requirement_text.value,
                            model="o3-mini"
                        )
                        print(f"分析完成，輸出資料夾：{output_folder}\n")
                    except Exception as e:
                        print(f"分析 {os.path.basename(path)} 時發生錯誤：{e}\n")
                else:
                    print(f"跳過：{os.path.basename(path)}\n")
    
    def on_cancel_clicked(b):
        with output_area:
            print("已取消分析。")
    
    confirm_button.on_click(on_confirm_clicked)
    cancel_button.on_click(on_cancel_clicked)
    
    # 組合所有 widget 並顯示
    display(widgets.VBox([
        widgets.Label("找到以下 Markdown 檔案（每個檔案可個別設定）："),
        files_vbox,
        widgets.HBox([confirm_button, cancel_button]),
        output_area
    ]))


In [ ]:
input_folder = ".\inputs"
output_folder = ".\outputs"
batch_md_analysis(input_folder, output_folder)